In [ ]:
import pickle
import csv
import random

In [ ]:
def load_pickle(filename):
    infile = open(filename,'rb')
    opened_file = pickle.load(infile)
    infile.close()
    return opened_file

In [ ]:
def shortlist_text_top_10(sorted_terms):
    #function to shortlist / remove redundancy from the sorted top-n result
    list_all_terms=[x[0].split() for x in sorted_terms]
    len_index = {}
    
    #make list of index with key length of words
    for i,item in enumerate(list_all_terms):

        if len(item) not in len_index:
            len_index[len(item)] = [i]
        else:
            len_index[len(item)].append(i)
    
    begins_with = ['the','a','an','of']
    ends_with = ['is']

    remove_begin = []
    idx_begins_ends = []
    for i,x in enumerate(list_all_terms):
        if x[0] in begins_with:
            remove_begin.append(x[1:])
            idx_begins_ends.append(i)
        else:
            remove_begin.append(x)
                       
    remove_begin_end = []
    for i,x in enumerate(remove_begin):
        if x[-1] in ends_with:
            remove_begin_end.append(x[:-1])
            idx_begins_ends.append(i)
        else:
            remove_begin_end.append(x)
    
    
    #remove_the = [x[1:] if x[0] in begins_with else x for x in list_all_terms] #Remove 'the','a','an' in candidate terms
    
    to_be_remove = []
    for j,item in enumerate(remove_begin_end):
        for i in range(len(remove_begin_end)):
            if (item == remove_begin_end[i]) & (i!=j):
                if i in idx_begins_ends:
                    to_be_remove.append(i)
    
    to_be_remove = sorted(set(to_be_remove)) #for keeeping the index number for items to be remove
            
    sorted_length = sorted(len_index.keys())
    max_len = sorted_length[-1]
    
    keep_counter = 0

    for i,candidate in enumerate(list_all_terms):
        if i not in to_be_remove:
            keep_counter = keep_counter + 1
            if i!=0:
                len_current = len(candidate)
                # check previous index which lenght is lower and not in to_be_remove list
                if len_current!=1:
                    for lower_len in range(1,len_current):
                        lower_len_indexes = len_index.get(lower_len,[]) #get indexes for lower length terms

                        for check_index in lower_len_indexes:
                            if (check_index<i) & (check_index not in to_be_remove):
                                delete_flag = True
                                for check_term in list_all_terms[check_index]:
                                    if check_term in candidate:
                                        delete_flag=delete_flag & True
                                    else:
                                        delete_flag = delete_flag & False

                                if delete_flag== True: #if the terms or multi terms are in the multi terms then flag to be deleted
                                    to_be_remove.append(check_index)
                                    keep_counter = keep_counter - 1

                #Check previous index which length is higher and not in to_be_remove list
                if len_current!=max_len:
                    for higher_len in range(len_current+1,max_len+1):
                        higher_len_indexes = len_index.get(higher_len,[])

                        for check_index in higher_len_indexes:
                            if (check_index<i) & (check_index not in to_be_remove):
                                delete_flag = True
                                previous_term = list_all_terms[check_index] #previous index term with higher length
                                for curr_term in candidate:
                                    if curr_term in previous_term:
                                        delete_flag = delete_flag & True
                                    else:
                                        delete_flag = delete_flag & False

                                if delete_flag == True:
                                    to_be_remove.append(i)
                                    keep_counter = keep_counter-1
                                    
        if keep_counter == 10:
            break
       
    keep_terms = [sorted_terms[x] for x in range(len(list_all_terms)) if x not in to_be_remove][0:10]
    
    
    return keep_terms #Index of the terms being kept

In [ ]:
#load data related to tag
tags = load_pickle('Data/tagged_tags_dictionary_withsents_film(available_in_index).pkl')
mapped_tags = load_pickle('Data/mapped_tags_to_group_film(available_in_index).pkl')

In [ ]:
#load saved sorted terms by log likelihood
sorted_terms_unigram = load_pickle('Data/sorted_terms/sorted_terms_tag_single_shingle_unigram_top30.pkl')
sorted_terms_unigram_multi = load_pickle('Data/sorted_terms/sorted_terms_tag_multi_shingle_unigram_top30.pkl')

sorted_terms_bigram = load_pickle('Data/sorted_terms/sorted_terms_tag_single_shingle_max2_top30.pkl')
sorted_terms_bigram_multi = load_pickle('Data/sorted_terms/sorted_terms_tag_multi_shingle_max2_top30.pkl')

sorted_terms_4gram = load_pickle('Data/sorted_terms/sorted_terms_tag_single_shingle_max4_top30.pkl')
sorted_terms_4gram_multi = load_pickle('Data/sorted_terms/sorted_terms_tag_multi_shingle_max4_top30.pkl')

for key in sorted_terms_unigram_multi:
    sorted_terms_unigram[key] = sorted_terms_unigram_multi[key]
    sorted_terms_bigram[key] = sorted_terms_bigram_multi[key]
    sorted_terms_4gram[key] = sorted_terms_4gram_multi[key]

In [ ]:
#select random adjective tags
adjective_tags = random.sample(mapped_tags['adjective'],30)
adjective_tags

In [ ]:
#select random non-adjective tags
mapped_non_adjective = []
for key in mapped_tags:
    if key != 'adjective':
        mapped_non_adjective = mapped_non_adjective + mapped_tags[key]
non_adjective_tags = random.sample(mapped_non_adjective,50)
non_adjective_tags

In [ ]:
# Write the top10 results for adjective and non adjective to tsv files for analysis of synonym

#out_file1 = open("Data/synonym_analysis/adjective_sorted_terms_random.tsv", 'wt')
out_file2 = open("Data/synonym_analysis/non_adjective_sorted_terms_random.tsv", 'wt') 

#with open("Data/sorted_terms/top_5_terms_per_tag(above_5).tsv", 'wt') as out_file:
#tsv_writer1 = csv.writer(out_file1, delimiter='\t')
tsv_writer2 = csv.writer(out_file2, delimiter='\t')
#tsv_writer1.writerow(["Tag"] + ['rank'+str(i) for i in range(1,11)])
tsv_writer2.writerow(["Tag"] + ['rank'+str(i) for i in range(1,11)])

for i,tag in enumerate(adjective_tags):
    sorted_terms_tag = shortlist_text_top_10(sorted_terms_4gram[tag])
    #tsv_writer1.writerow([tag] + [x[0] for x in sorted_terms_tag])
#out_file1.close()
for i,tag in enumerate(non_adjective_tags):
    sorted_terms_tag = shortlist_text_top_10(sorted_terms_4gram[tag])
    tsv_writer2.writerow([tag] + [x[0] for x in sorted_terms_tag])
out_file2.close()